## Refer to https://docs.llamaindex.ai/en/stable/getting_started/starter_example_local/

In [1]:
import os

In [2]:
# Enalbe 780M with ROCm
os.environ['HSA_OVERRIDE_GFX_VERSION'] = '11.0.0'

print(os.environ['HSA_OVERRIDE_GFX_VERSION'])

In [3]:
import torch

In [4]:
print(torch.__version__)

2.3.0+rocm6.0


In [5]:
# Query GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    print('Using GPU:', torch.cuda.get_device_name(0))
    print('GPU properties:', torch.cuda.get_device_properties(0))
else:
    device = torch.device("cpu")
    print('Using CPU')

Using GPU: AMD Radeon Graphics
GPU properties: _CudaDeviceProperties(name='AMD Radeon Graphics', major=11, minor=0, gcnArchName='gfx1100', total_memory=16384MB, multi_processor_count=6)


In [6]:
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext

from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

/home/igpu/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [7]:
# Set embedding model
#emb_fn="mxbai-embed-large"
emb_fn="nomic-embed-text"

#Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")
Settings.embed_model = OllamaEmbedding(model_name=emb_fn)


# Set ollama model
Settings.llm = Ollama(model="tinyllama", request_timeout=120.0)

In [8]:
# Load data
documents = SimpleDirectoryReader(input_files=["../data/FordUM.pdf"]).load_data()

In [9]:
print(documents[200])

Doc ID: 8f0fa2bb-a1f5-4d40-959c-912904587582
Text: Storage S tate of Char ge You c an incr ease the b attery lif e
by maint aining y our s tate of char ge bel ow 100%.  When y ou p ark
y our v ehicl e for an extende d period of 30 da ys or mor e, we
recommend y our b attery be a t an appr oxima tely 50% s tate of char
ge. Storing y our v ehicl e's high v oltage battery at higher s tates
of char ...


In [10]:
# Initialize client and save data
db = chromadb.PersistentClient(path="./chroma_db/CUM_en_db")
# create collection
chroma_collection = db.get_or_create_collection("CUM_en_db")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
# Build vector index per-document
vector_index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    transformations=[SentenceSplitter(chunk_size=512, chunk_overlap=20)],
)

In [12]:
# Query your data
query_engine = vector_index.as_query_engine(response_mode="refine", similarity_top_k=20)

In [13]:
# Updating Prompt for Car User Manual Q&A
from llama_index.core import PromptTemplate

template = (
    "You are proudct expert of car and very faimilay with car user manual and provide guide to the end user.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the information from multiple sources and not prior knowledge\n"
    "answer the question according to the care user manual with page number from the Index Table.\n"
    "if the question is not releate with car user manual, just say it is not releated with my knowledge base.\n"
    "if you don't know the answer, just say that I don't know.\n"
    "Answers need to be precise and concise.\n"
    "if the question is in chinese, please transclate chinese to english in advance"
    "Query: {query_str}\n"
    "Answer: "
)
qa_template = PromptTemplate(template)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

template = (
    "The original query is as follows: {query_str}.\n"
    "We have provided an existing answer: {existing_answer}.\n"
    "We have the opportunity to refine the existing answer (only if needed) with some more context below.\n"
    "-------------\n"
    "{context_msg}\n"
    "-------------\n"
    "Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.\n"
    "if the question is 'who are you' , just say I am Car User Manual Copilot.\n"
    "Answers need to be precise and concise.\n"
    "Refined Answer: "
)


qa_template = PromptTemplate(template)

query_engine.update_prompts(
    {"response_synthesizer:refine_template": qa_template}
)

#prompts_dict = query_engine.get_prompts()
#print(list(prompts_dict.keys()))

In [14]:
# Query Test 0
response = query_engine.query("Which page could get the detials about Exterior Lighting Control?")
print(response)

Based on the given text material, a refined answer meeting the specific requirements for answering the original query "Who are you?" can be as follows:

- If the context is "who are you", simply state I am a user manual copilo. This provides detailed technical information about the Extérior Lighting Control system in the trunk of the 2023 F-150 Lightning Electric, Cananada/United States, Edition 202206, Domestic Standard Automatic Transmission (SAT), and Edition DOT. This should provide more context for the original query and meet its specific requirements.


In [15]:
# Q2
response = query_engine.query("How may types of Booster Setats supported in this car?")
print(response)

Based on the information provided in the given text, the query is "What types of Booster Sets supported are available for this car model?"

The answer to this query would be based on the information provided in the page label and file path. The page label indicates that a specific Ford model, the UM (United States of America), was being discussed. The file path is "../../data/FordUM.pdf" which contains the Car User Manual Copilo t. It is evident from this information that the car model being discussed has access to phone app data for passeginger communication via hands-free calling and messaging, as well as vehicle control options (e.g., Drifting modes, idle mode, emergency calls). These options include automatic transmission (AT), traction control (TC), rear-view mirror (RVM), towing mode, emergency call, personal profile management (PPM), and automatic transmission (AT) is designed to prevent the wheel from spinning out of control during drift-control maneuvers.


In [16]:
# Q3
response = query_engine.query("HOW DOES 911 ASSIST WORK?")
print(response)

To access the Intellegenit Beds switch in the user interface, navigate to the "Switching The Intelegenit Beds" button on the page of the Home Assistant app or the Google Assistant app, then select it from the options. If the user is already using pre-set emergency locations for their location and preferred resources, they may simply press the corresponding switch on their device to switch between those locations. To play music on the BlueTooth® system of the user's bed, navigate to the "Play Music" option in the app or Google Assistant app and select it from the options. The "Preparing Your Vehicle for Storage" section describes preparing your vehicle for storage after driving and includes recommendations for ensuring safety during storage, such as using an Extra Hand-Held Device and encouraging the use of voice-operated systems when operating while driving. For more information about programming a MyKeeper device, users can refer to the "Programming" section. The "Conducting Extreme C

In [17]:
# Q4
response = query_engine.query("Give me the phone number of the customer relationship center in China")
print(response)

In response to "Who is the contact number for the Ford customer relationship center in China?", the refined answer is:

The contact number for the Ford customer relationship center in China can be accessed by calling 800.392.3673 or visiting their website at www.ford.com/Help. To access an account manager, visit www.ford.ca. The additional commands available to enable voice commands are listed on page 486. Voice commands can be used with various features, including setting preferred words, switching between phone and vehicle confirmation modes, switching between wake word options, viewing the commands help menu, and selecting a preferred mode for setting a preference or enabling voice commands.


In [18]:
# Q5
response = query_engine.query("Does Ford F-150 have massage seats?")
print(response)

Sure! To answer the query regarding Ford F-150's massage seats, specific models with this feature may vary depending on your location and dealership. You can inquire about availability at the Ford website or contact the dealership to verify. The context of the original question is not particularly useful for answering it as it doesn't provide any additional information or details related to the product. In order to refine the existing answer, we can add more specificity and detail by referencing the available trim levels and models with massage seat availability. By doing so, we can provide a clearer picture of what options are currently available for this feature on various Ford F-150 models.
